In [2]:
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import os
import itertools

from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from sklearn.utils import shuffle

Using TensorFlow backend.


In [4]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="char-embeddings.txt",
               word2vec_output_file="gensim_char-embeddings.txt")

In [5]:
char_embeddings = KeyedVectors.load_word2vec_format("../gensim_char-embeddings.txt", binary=False)
char_vectors = char_embeddings.wv

In [29]:
glove_model = KeyedVectors.load_word2vec_format("../gensim_glove_vectors25.txt", binary=False)
vectors = glove_model.wv

In [13]:
def glove_char_embeddings(tokens):
    chars_num = 150;
    dim = 300
    
    arr = np.zeros(dim)
    
    for word in tokens:
        for char in word:
            try:
                arr = np.add((arr, char_vectors[char]))
            except:
                pass
    
    return np.array(arr)

In [87]:
def glove_vects(tokens):
    
    arr = np.zeros(25)
    
    for tok in tokens:
        try:
            arr = np.add(arr, vectors[tok])
        except:
            pass
        
    return np.array(arr)

In [14]:
def preprocessTweet(tweet):
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(tweet)
    
    return glove_char_embeddings(tokens)

In [20]:
def load_document(data_location, htf):
    tweets = []
    labels = []
    
    for line in open(os.path.join(data_location, htf)).readlines():
        line_split = line.strip().split('\t')
        tweets.append(preprocessTweet(line_split[1]))
        labels.append(int(line_split[2]))

    return {'tweets': tweets, 'labels': labels}

In [21]:
def create_data(data_location):
    ht_files = sorted(os.listdir(data_location))

    Xs = []
    ys = []
    ht_list = []
    for htf in ht_files:
        ht_dict = load_document(data_location, htf)

        ht_list.append(htf)
        ys.append(ht_dict['labels'])
        Xs.append(ht_dict['tweets'])
        
    return Xs, ys, ht_list

# Ovaj

In [66]:
Xs, ys, ht_list = create_data('../trial_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = np.array(ys[0])

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# class_weight = {0: 1., 1: 8}

model.fit(x_train, to_categorical(y_train),
          epochs=10,
          batch_size=64)

# score = model.evaluate(x_test, y_test, batch_size=1)
# print(score)
y_pred = model.predict([x_test], verbose=1)
print(y_pred)
# print(np.hstack((y_test.reshape(y_test.shape[0], 1), y_pred)))

Epoch 1/10
550/550 [==============================] - 2s 3ms/step - loss: 0.6899 - acc: 0.9273
Epoch 2/10
550/550 [==============================] - 0s 52us/step - loss: 0.6822 - acc: 0.9273
Epoch 3/10
550/550 [==============================] - 0s 42us/step - loss: 0.6749 - acc: 0.9273
Epoch 4/10
550/550 [==============================] - 0s 35us/step - loss: 0.6674 - acc: 0.9273
Epoch 5/10
550/550 [==============================] - 0s 33us/step - loss: 0.6601 - acc: 0.9273
Epoch 6/10
550/550 [==============================] - 0s 40us/step - loss: 0.6530 - acc: 0.9273
Epoch 7/10
550/550 [==============================] - 0s 39us/step - loss: 0.6459 - acc: 0.9273
Epoch 8/10
550/550 [==============================] - 0s 40us/step - loss: 0.6390 - acc: 0.9273
Epoch 9/10
550/550 [==============================] - 0s 37us/step - loss: 0.6322 - acc: 0.9273
Epoch 10/10
110/110 [==============================] - 1s 6ms/step
[[0.5440488  0.45595118]
 [0.5440488  0.45595118]
 [0.5440488  0.45595

In [63]:
Xs, ys, ht_list = create_data('../train_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

max_features = 1024

model = Sequential()
# model.add(Embedding(input_dim=25, output_dim=256))
model.add(LSTM(128, input_shape = (len_of_seq, 25), return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)

y_pred = model.predict_classes(x_test, verbose=1)
print(y_pred)
# score = model.evaluate(x_test, y_test, batch_size=16)

ValueError: Input 0 is incompatible with layer lstm_16: expected ndim=3, found ndim=2

In [54]:
Xs, ys, ht_list = create_data('../train_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])
x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)
y_pred = regressor.predict(x_test)
print(np.hstack((y_test, y_pred)))

ValueError: Error when checking input: expected lstm_5_input to have 3 dimensions, but got array with shape (11186, 25)

In [168]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 300
timesteps = 150
num_classes = 2

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Xs, ys, ht_list = create_data('../trial_data')

x_train = np.array([*itertools.chain.from_iterable(Xs[1:])])
y_train = np.array([*itertools.chain.from_iterable(ys[1:])])

x_train, y_train = shuffle(x_train, y_train)

x_test = Xs[0]
y_test = ys[0]

y_train[y_train == 2] = 1
y_test[y_test == 2] = 1

model.fit(x_train, y_train,
          batch_size=64, epochs=15)

print(np.hstack((y_test, model.predict(x_test))))

Epoch 1/15
550/550 [==============================] - 16s 30ms/step - loss: 0.6649 - acc: 0.9200
Epoch 2/15
550/550 [==============================] - 4s 7ms/step - loss: 0.4807 - acc: 0.9273
Epoch 3/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2768 - acc: 0.9273
Epoch 4/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2703 - acc: 0.9273
Epoch 5/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2607 - acc: 0.9273
Epoch 6/15
550/550 [==============================] - 4s 6ms/step - loss: 0.2612 - acc: 0.9273
Epoch 7/15
550/550 [==============================] - 4s 7ms/step - loss: 0.2611 - acc: 0.9273
Epoch 8/15
550/550 [==============================] - 4s 6ms/step - loss: 0.2608 - acc: 0.9273
Epoch 9/15
550/550 [==============================] - 3s 6ms/step - loss: 0.2613 - acc: 0.9273
Epoch 10/15
550/550 [==============================] - 3s 6ms/step - loss: 0.2615 - acc: 0.9273
Epoch 11/15
550/550 [==========================